In [45]:
import csv
import json
import sys,os
rel_path = os.path.realpath('./data')
rel_path += '/'

In [46]:
class CSVTable():

    # Change to wherever you want to save the CSV files.

    def __init__(self, table_name, table_file, key_columns):
        '''
        Constructor
        :param table_name: Logical names for the data table.
        :param table_file: File name of CSV file to read/write.
        :param key_columns: List of column names the form the primary key.
        '''
        self.table_name = table_name
        self.table_file = rel_path + table_file
        self.key_columns = key_columns
        self.data = []
    
    def __str__(self):
        '''
        Pretty print the table and state.
        :return: String
        '''
        return json.dumps(self.data, indent=2)
        
    def load(self):
        '''
        Load information from CSV file.
        :return: None
        '''
        try:
            with open(self.table_file, mode = 'r') as csv_file:
                csv_reader = csv.DictReader(csv_file)
                for i in csv_reader:
                    self.data.append(i)
                

        except IOError:
            print ("The datatable file doens't exit")
        
        for i in self.key_columns:
            if i not in self.data[0]:
                raise ValueError("The primary keys are invalid")

    def find_by_primary_key(self, string_set, fields=None):

        if len(string_set) != len(self.key_columns):
            raise ValueError("The primary keys inputed didn't match the key columns")
        result = []
        if fields == None:
            for i in self.data:
                count_match = 0
                for j in range(0,len(string_set)):
                    if i[self.key_columns[j]] == string_set[j]:
                        count_match += 1
                if count_match == len(self.key_columns):
                    result.append(i)
        else:
            
            for j in fields:
                if j not in self.data[0].keys():
                    raise ValueError("The fields don't match the database column types")
            
            unwanted = self.data[0].keys() - fields
            for i in self.data:
                count_match = 0
                for j in range(0,len(string_set)):
                    if i[self.key_columns[j]] == string_set[j]:
                        count_match += 1
                if count_match == len(self.key_columns):
                    for unwanted_key in unwanted:
                        temp = i.copy()
                        del temp[unwanted_key]
                    result.append(temp)

        return result        

    def find_by_template_top(self):
        result = set()

        for i in self.data:
            if int(i["yearID"]) >= 1960 and int(i["AB"]) >= 200:
                result.add(i['playerID'])
        
        return result




In [47]:
csvt_batting = CSVTable("Batting", "Batting.csv",["playerID"])
csvt_batting.load()

csvt = CSVTable("People", "People.csv", ["playerID"])
csvt.load()            

In [50]:
player_set = csvt_batting.find_by_template_top()

player_dict = {}
for i in player_set:
    string_set =[]
    string_set.append((i))
    r = csvt_batting.find_by_primary_key(string_set,["AB","H"])
    total_ab = 0
    total_h = 0
    for j in r:
        total_ab += int(j["AB"])
        total_h += int(j["H"]) 
    player_dict[i] = float(total_h)/total_ab

player_dict_sorted = sorted(player_dict.items(),key = lambda x:x[1],reverse = True)

final_result = []
for i in player_dict_sorted:
    string_set = []
    string_set.append(i[0])
    r = csvt.find_by_primary_key(string_set)
    final_result.append(r)
    
    if len(final_result) == 10:
        break
print(json.dumps(final_result, indent=2))

[
  [
    {
      "playerID": "willite01",
      "birthYear": "1918",
      "birthMonth": "8",
      "birthDay": "30",
      "birthCountry": "USA",
      "birthState": "CA",
      "birthCity": "San Diego",
      "deathYear": "2002",
      "deathMonth": "7",
      "deathDay": "5",
      "deathCountry": "USA",
      "deathState": "FL",
      "deathCity": "Inverness",
      "nameFirst": "Ted",
      "nameLast": "Williams",
      "nameGiven": "Theodore Samuel",
      "weight": "205",
      "height": "75",
      "bats": "L",
      "throws": "R",
      "debut": "4/20/1939",
      "finalGame": "9/28/1960",
      "retroID": "willt103",
      "bbrefID": "willite01"
    }
  ],
  [
    {
      "playerID": "gwynnto01",
      "birthYear": "1960",
      "birthMonth": "5",
      "birthDay": "9",
      "birthCountry": "USA",
      "birthState": "CA",
      "birthCity": "Los Angeles",
      "deathYear": "2014",
      "deathMonth": "6",
      "deathDay": "16",
      "deathCountry": "USA",
      "deathSt

In [2]:
import pymysql
import json

cnx = pymysql.connect(host= "localhost",
                      user= "root",
                      password= "zhangchi25",
                      db="lahman2017n",
                      charset='utf8mb4',
                      cursorclass=pymysql.cursors.DictCursor)
cursor = cnx.cursor()
q = "ELECT \
Batting.playerID, \
(SELECT People.nameFirst FROM People WHERE People.playerID=Batting.playerID) as first_name, \
(SELECT People.nameLast FROM People WHERE People.playerID=Batting.playerID) as last_name, \
sum(Batting.h)/sum(batting.ab) as career_average, \
sum(Batting.h) as career_hits, \
sum(Batting.ab) as career_at_bats,\
min(Batting.yearID) as first_year, \
max(Batting.yearID) as last_year \
FROM \
Batting \
GROUP BY \
playerId \
HAVING \
career_at_bats > 200 AND last_year >= 1960 \
ORDER BY \
career_average DESC \
LIMIT 10;"

cursor.execute(q)
r = cursor.fetchall()
print(json.dumps(r, indent=2))

ProgrammingError: (1064, "You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near 'ELECT Batting.playerID, (SELECT People.nameFirst FROM People WHERE People.player' at line 1")